In [1]:
import tensorflow as tf
import numpy as np
from tf_cnnvis import *
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
IMG_SIZE_PX = 128
SLICE_COUNT = 28

n_classes = 2
batch_size = 1
# test_data = np.load('D:/NORMvsSAH/Data/NORM&SAH_Data_Folds/Fold1/Test.npy')#for 50
# test_data = test_data[10:11]
test_data = np.load('D:/NORMvsSAH/Data/224/Fold1/Check224.npy')####For 224
test_data = test_data [0:1]
@ops.RegisterGradient("Customlrn")
def _CustomlrnGrad(op, grad):
    return grad
@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(grad > 0.0, gen_nn_ops.relu_grad(grad, op.outputs[0]), tf.zeros(grad.get_shape()))

In [2]:

# @ops.RegisterGradient("GuidedRelu")
# def _GuidedReluGrad(op, grad):
#     return tf.where(grad > 0.0, gen_nn_ops.relu_grad(grad, op.outputs[0]), tf.zeros(grad.get_shape()))

session = tf.Session()
eval_graph = tf.Graph()
with eval_graph.as_default():
    with eval_graph.gradient_override_map({'Relu': 'GuidedRelu'}):
        x = tf.placeholder('float')
        y = tf.placeholder('float')
        keep_rate = 1    
        Input = tf.reshape(x, [1, SLICE_COUNT, IMG_SIZE_PX, IMG_SIZE_PX, 1])

        with tf.name_scope('Conv1'):
            Filter1 = tf.get_variable(name='Filter1', shape=[3,3,3,1,32]) 
            Bias1 = tf.get_variable(name='Bias1', shape=[32]) 
            conv1 = tf.nn.conv3d(input = Input,filter = Filter1,strides=[1, 1, 1, 1, 1],padding='SAME')
            Conv1 = tf.nn.relu(conv1 + Bias1)
    #     print("Conv1 = ", Conv1.get_shape())
    #######LAYER 2 CONV2 =============================================================

    #######LAYER 2 POOL2 =============================================================
        with tf.name_scope('Pool1'):
            Pool1 = tf.nn.max_pool3d(input = Conv1,  ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

        #         print("Conv1 = ", Conv1.get_shape())

    ######LAYER 3 CONV3 =============================================================
        with tf.name_scope('conv2'):
            Filter2 = tf.get_variable(name='Filter2', shape=[3,3,3,32,64])
            Bias2 = tf.get_variable(name='Bias2', shape=[64])
            conv2 = tf.nn.conv3d(input = Pool1, filter = Filter2, 
                            strides=[1, 1, 1, 1, 1], 
                            padding='SAME')
            Conv2 = tf.nn.relu(conv2 + Bias2)  
    #     print("Conv1 = ", Conv1.get_shape())
        with tf.name_scope('Pool2'):
            Pool2 = tf.nn.max_pool3d(input = Conv2, 
                         ksize=[1, 2, 2, 2, 1],
                         strides=[1, 2, 2, 2, 1], 
                         padding='SAME')   
    #######LAYER 4 CONV4 =============================================================
        with tf.name_scope('conv3'):
            Filter3 = tf.get_variable(name='Filter3', shape=[3,3,3,64,128])
            Bias3 = tf.get_variable(name='Bias3', shape=[128]) 
            conv3 = tf.nn.conv3d(input = Pool2, 
                            filter = Filter3, 
                            strides=[1, 1, 1, 1, 1], 
                            padding='SAME')
            Conv3 = tf.nn.relu(conv3 + Bias3) 
#     print("Conv1 = ", Conv1.get_shape())
    #######LAYER 4 POOL4 =============================================================
        with tf.name_scope('Pool3'):
            Pool3 = tf.nn.max_pool3d(input = Conv3, 
                         ksize=[1, 2, 2, 2, 1],
                         strides=[1, 2, 2, 2, 1], 
                         padding='SAME')
     #     print("Conv1 = ", Conv1.get_shape())
    
        with tf.name_scope('fc1') as scope:
            FilterFc1 = tf.get_variable(name='FilterFc1', shape=[131072, 1024]) #for 224-401408 and for 50-25088 and
            BiasFc1 = tf.get_variable(name='BiasFc1', shape=[1024])             # for 128- 131072
            FlatFc1 = tf.reshape(Pool3, [-1,131072])  
            OutFc1 = tf.nn.relu(tf.matmul(FlatFc1, FilterFc1) + BiasFc1)
            DropOutFc1 = tf.nn.dropout(OutFc1, keep_rate)
# print("Conv1 = ", Conv1.get_shape())
        with tf.name_scope('Out') as scope:
            FilterFc2 = tf.get_variable(name='FilterFc2', shape=[1024, n_classes])
            BiasFc2 = tf.get_variable(name='BiasFc2', shape=[n_classes])
            prediction = tf.matmul(DropOutFc1, FilterFc2) + BiasFc2    
#     print("Conv1 = ", Conv1.get_shape())
    


        with tf.name_scope('Loss'):
            cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) ) 
        with tf.name_scope('SGD'):
            optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost) 
            init = tf.global_variables_initializer()




Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [3]:

# saver = tf.train.Saver()       

outn = []
with tf.Session(graph=eval_graph) as sess:
    sess.run(init)
    saver = tf.train.Saver()
#     saver.restore(sess, "D:/NORMvsSAH/Metadata/224/Layer3264128/Fold1/model.ckpt")  #  for 224
#     saver.restore(sess, "D:/NORMvsSAH/MetaData/Layer3264128/Fold1/Threshold/model.ckpt") # for 50
    saver.restore(sess, "D:/NORMvsSAH/Metadata/128/Layer3264128/Fold1/Threshold/70/model.ckpt") # for 128    
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    probabilities = tf.nn.softmax(prediction)
    activation=[]
    
    op_tensor =  Pool1
    
    for data in test_data:
        img = data[0][0]/255
        img[img<.35]=0                
        img_ind = 1
        x = x
        
        feed_dict ={x:  img}
#         op_tensor =  Conv1
        activation=(sess.run(op_tensor, feed_dict = feed_dict))

        print('op_tensor',np.shape(op_tensor))
        print('activation',np.shape(activation))


INFO:tensorflow:Restoring parameters from D:/NORMvsSAH/Metadata/128/Layer3264128/Fold1/Threshold/70/model.ckpt


In [4]:

# saver = tf.train.Saver()       

outn = []
with tf.Session(graph=eval_graph) as sess:
    sess.run(init)
    saver = tf.train.Saver()
#     saver.restore(sess, "D:/NORMvsSAH/Metadata/224/Layer3264128/Fold1/model.ckpt")  #  for 224
#     saver.restore(sess, "D:/NORMvsSAH/MetaData/Layer3264128/Fold1/Threshold/model.ckpt") # for 50
    saver.restore(sess, "D:/NORMvsSAH/Metadata/128/Layer3264128/Fold1/Threshold/70/model.ckpt") # for 128    
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    probabilities = tf.nn.softmax(prediction)
    for data in test_data:
        img = data[0][0]/255
        img[img<.35]=0        
        img_ind = 1
        x = x
        out=[]
        feed_dict ={x:  img}
        op_tensor =  Conv3
        activation = sess.run(op_tensor, feed_dict = feed_dict)
        for i in range(4):
            activation = np.sum(activation, axis=0)
        inds = np.argpartition(activation, -4)[-4:]
        tensor_shape = op_tensor.get_shape().as_list()
        
        for ind in inds:
            reconstruct = [tf.gradients(tf.transpose(tf.transpose(op_tensor[:,:, :, :, ind])), x)[0] ]
            outn.extend(sess.run(reconstruct, feed_dict = feed_dict))
        
        print('op_tensor',np.shape(op_tensor))
        print('activation',np.shape(activation))
        print('inds',inds)
        print('outn',np.shape(outn))

        

INFO:tensorflow:Restoring parameters from D:/NORMvsSAH/Metadata/128/Layer3264128/Fold1/Threshold/70/model.ckpt


In [5]:
######For ACTIVATION VISUALIZATION
from matplotlib.pyplot import imsave
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
import cv2
cmap = plt.cm.bone
from math import ceil, sqrt
# from scipy.misc import imsave
from matplotlib.pyplot import imsave

activationx=activation[0][7].reshape(1,112,112,32)
act_shape = np.shape(activationx)
activations = activationx

act_shape
if len(act_shape) == 2:
    grid_activations = [np.expand_dims(image_normalization(convert_into_grid(im[:,np.newaxis,np.newaxis,np.newaxis], padding=0)), axis = 0) for im in activations]
else:
    activations = [np.expand_dims(im, axis = 3) for im in np.transpose(activations, (3, 0, 1, 2))]
    activations = _im_normlize(activations)
    grid_activations = _images_to_grid(activations)
for i in range(len(grid_activations)):
    imsave(( "D:/actn_vis.png"), grid_activations[i][0,:,:,0], format = "png", cmap=cmap)

In [ ]:
######For DECONV VISUALIZATION
from matplotlib.pyplot import imsave
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
import cv2
out=[]
cmap = plt.cm.bone
for i in range(len(outn)):
    im=outn[i][14]
#     backtorgb = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)    
    out.append(im.reshape(1,IMG_SIZE_PX,IMG_SIZE_PX,1))
images = _im_normlize(out)
grid_images = _images_to_grid(images)
for i in range(len(grid_images)):
    if grid_images[i].shape[-1] == 1:
        imsave(("D:/DECO_Vis.png"), grid_images[i][0,:,:,0], format = "png", cmap=cmap)
        plt.imshow(grid_images[i][0,:,:,0],cmap=cmap )
        plt.show()
#         imsave(("D:/DECO_Vis.png"), grid_images[i][0,:,:,0], format = "png")
    else:
        imsave(( "D:/DECO_Vis.png"), grid_images[i][0], format = "png")

In [19]:
np.shape(activation[0][7].reshape(11))

(112, 112, 32)